In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm
from BertClassifier import BertClassifier
from DataSet import Dataset

model = BertClassifier()
model = nn.DataParallel(model)
model.load_state_dict(torch.load("state_model.pt"))

In [ ]:
import pandas as pd

comment_text_df = pd.read_csv('commentData.csv')
predict_df = pd.read_csv('comment_test.csv')

# bbc_text_df.head()



df = pd.DataFrame(comment_text_df)
df_predict = pd.DataFrame(predict_df)

display(df)
display(df_predict)

np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

#1780 222 223

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        for test_input, test_label in test_dataloader:
              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)
              output = model(input_id, mask)
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)

In [ ]:
torch.cuda.empty_cache()